# Multiple fit code

In [160]:
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import factorial
from scipy import optimize
from matplotlib.pyplot import cm
from lmfit import Model
from matplotlib.gridspec import GridSpec
from scipy.stats import poisson as pois
from scipy.stats import norm

## Multiple loading (load + prep)

In [161]:
def multipleLoad(path='db/',bin_width=651,nbins=100):
    
    def list_files(path):
        listFile=[]
        for root, dirs, files in os.walk(path):
            for file in files:
                listFile.append(os.path.join(root, file))
        return listFile
                
    
    file_list=list_files(path)
    InfoDataset=[]
    
    for file in file_list:
        
        meta = pd.read_excel(file,sheet_name=0,header=None)
        freq = pd.read_excel(file,sheet_name=1)             # frequecies
        fft  = pd.read_excel(file,sheet_name=2)             # power
    
        data = pd.DataFrame({'freq':freq[1]})
    
        col = 0
        for col_name in fft.columns: # load all the subruns
            if col > 0:
                data[f'fft{col-1}'] = fft[col_name]
            col += 1
        
        common_column = 'freq'
        
        # List of columns to melt
        columns_to_melt = data.columns[1:]  # Exclude the first column
        common_column = 'freq'
        columns_to_melt = data.columns[1:]  # Exclude the first column

        # Flatten the DataFrame
        data_flat = pd.melt(data, id_vars=common_column, value_vars=columns_to_melt, 
                            var_name='subRun', value_name='fft')

        #cavity frequency and number of files per run
        center = meta[1][3]
        length = meta[1][8]
        
        N = length*2731 #N=1365500 if length=500
        
        # select window of 2*nbins bins around center
        # default is to select 200 bins of 651 Hz
        mask = (data_flat['freq']>center-bin_width*nbins) & (data_flat['freq']<center+bin_width*nbins)
        cavdata = data_flat[mask].reset_index(drop=True)
        
        # scale data to yottowat
        minW = np.min(cavdata["fft"].copy()) # minimum power in the cavity
        
        # In general, the average measured power should be known and equal to the noise temperature of the system.
        # So we can rescale the data so that the power at the cavity frequency sia T_noise k_b B (W)
        ref = minW**(-1) * 3.5*1.38e-23*651/1e-24 #It is possibile to add an extra contribute to make them integers
        
        cavdata["fft"] = ref * cavdata["fft"]   #y' 
        
        # set weights
        #weights = cavdata"fft"]/np.sqrt(N)      # -> y'/sqrt(N)
        weights = np.sqrt(ref)*np.sqrt(cavdata["fft"])/np.sqrt(N)  #-> sqrt(sigma'/N)=ref*sqrt(y/N)
        
        Info={"name":file,"length":length,"center":center,"cavdata":cavdata,"weights":weights,"ref":ref}        
        InfoDataset.append(Info)
     
    return(InfoDataset)

## Multiple fit

### Fit utilities

In [195]:
def fit_bkg(freq, fft, weights, center, ref):
    # set fit model
    bkg_model = Model(bkg)
    ps = bkg_model.make_params(a={'value':center, 'min':center*0.999, 'max':center*1.01},
                               b=2e4,
                               c={'value':center, 'min':center*0.999, 'max':center*1.01},
                               d=2.2e4,
                               e=1e-2*np.sqrt(ref),
                               f=1e-12*ref)
    
    # execute fit
    result = bkg_model.fit(fft,x=freq,params=ps,weights=1/weights)
    
    return(result)

In [197]:
def fit_sig(freq, fft, weights, 
            signal,res_bkg,x_0,  mu_init = 1e-8, mu_vary = True):
    
    # take result of preliminary background fit to fix starting parameters
    p = res_bkg.best_values
    
    # set fit model
    sig_model = Model(signal)
    ps = sig_model.make_params(a ={'value':p['a'], 'vary':False},
                               b ={'value':p['b'], 'vary':False},
                               c ={'value':p['c'], 'vary':False},
                               d ={'value':p['d'], 'vary':False},
                               e ={'value':p['e'], 'vary':False},
                               f ={'value':p['f'], 'vary':False},
                               mu={'value':mu_init, 'min':0, 'vary':mu_vary},
                               x0={'value':x_0, 'vary':False},
                               s ={'value':16*651, 'vary':False}) # fixed value to 16 bins

    result = sig_model.fit(fft,x=freq,params=ps,weights=1/weights)
    
    return(result)

In [198]:
def gaussian(x,x0,s,mu):
    return mu * np.exp(-.5*((x-x0)/s)**2)
    
def maxwell(x,x0,s,mu):
    return mu * x**2/s**3 * np.exp(-.5*((x-x0)/s)**2)
    
def bkg(x,a,b,c,d,e,f):                                                   
    return e**2*abs(x-a+1j*b)**2/abs(x-c+1j*d)**2+f*(x-c)
    
def signal_gauss(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + gaussian(x,x0,s,mu)
    
def signal_maxwell(x,a,b,c,d,e,f,x0,s,mu):
    return bkg(x,a,b,c,d,e,f) + maxwell(x,x0,s,mu)

### Multiple fit function

In [199]:
def multipleFit(InfoDataset,fitSig=False,fitInfo={}):
    fitResult=[]
    for run in InfoDataset:
        
        bkg_result=fit_bkg(run["cavdata"]["freq"],run["cavdata"]["fft"],
                           run["weights"],run["center"],run["ref"])
        
        results={"run":run["name"],
                 "background_bestFit":bkg_result.params.valuesdict(),"background_residuals":bkg_result.residual}
        
        if fitSig:
            sig_result=fit_sig(run["cavdata"]["freq"],run["cavdata"]["fft"],run["weights"],
                               fitInfo["signalFunc"],bkg_result,
                               fitInfo["x0"],fitInfo["mu_init"],fitInfo["mu_vary"])
                               
            
            results={"run":run["name"],
                     "background":{"background_bestFit":bkg_result.params.valuesdict(),"background_residuals":bkg_result.residual},
                     "signal":{"signal_bestFit":sig_result.params.valuesdict(),"signal_residuals":sig_result.residual}}
        
        fitResult.append(results)
    
    return(fitResult)

## Fit test

In [202]:
InfoDataset=multipleLoad(path='db/')
fitInfo={"signalFunc":signal_gauss,"x0":0, "mu_init" :1e-8, "mu_vary": True}

In [203]:
fits=multipleFit(InfoDataset,True,fitInfo)